In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/tabular-playground-series-dec-2021/train.csv")
data.info

In [ ]:
data.describe()

In [ ]:
data['Cover_Type'].value_counts()

In [ ]:
data['Cover_Type']=data['Cover_Type'].apply(lambda x:x-1)

In [ ]:
data.drop('Id',axis=1,inplace=True)
data.dropna(inplace=True)
data.dropna(how="all",axis=1,inplace=True)

In [ ]:
data=data.sample(frac=1,ignore_index=True)

In [ ]:
trainx = data[data.columns[:-1]]
trainy = data['Cover_Type']

In [ ]:
cols = data.columns
del data

In [ ]:
from sklearn.preprocessing import StandardScaler

trainx = StandardScaler().fit_transform(trainx)

In [ ]:
from scipy.stats import pearsonr
corr = []
for i in range(trainx.shape[1]):
    corr.append(pearsonr(trainx[:,i],trainy)[0])

In [ ]:
import plotly.express as px
px.bar(corr)

In [ ]:
featureindex=[]
for i in range(len(corr)):
    if abs(corr[i])>=0.1:
        featureindex.append(i)

trainx = trainx[:,featureindex]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(trainx,trainy)

In [ ]:
testdata = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')

testdata.head()

In [ ]:
testdata.drop('Id',inplace=True,axis=1)

In [ ]:
testx = StandardScaler().fit_transform(testdata.values)
testx = testx[:,featureindex]

In [ ]:
predictions = clf.predict(testx)
predictions = [i+1 for i in predictions]

In [ ]:
sub['Cover_Type'] = predictions
sub.to_csv("submission.csv", index=False)